In [ ]:
!pip install datasets

In [ ]:
import json
import pandas as pd
import librosa
import numpy as np

import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,accuracy_score

from keras import layers,models,optimizers,losses
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

## Using pitch and fbank features - FNN


In [ ]:
train_data = pd.read_csv("./drive/Shareddrives/SWDA/swb1_d1/train_95.csv")
test_data = pd.read_csv("./drive/Shareddrives/SWDA/swb1_d1/test_95.csv")
#train_90,test_90 - 90 percentile data
#train_95,test_95 - 95 percentile data
#train_data,test_data - max pooling data

In [ ]:
X_train = train_data.iloc[:,-7:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,-7:-1]
y_test = test_data.iloc[:,-1]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
y_test=le.transform(y_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,
                                                 test_size=0.1,stratify=y_train)

In [ ]:
le.classes_.shape

(41,)

In [ ]:
import keras
input_shape=(6,)
model = models.Sequential()
model.add(Dense(128, activation='relu',input_shape=input_shape))
model.add(Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(41, activation='softmax'))
model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 128)               896       
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dropout_27 (Dropout)        (None, 64)                0         
                                                                 
 dense_47 (Dense)            (None, 128)               8320      
                                                                 
 dropout_28 (Dropout)        (None, 128)               0         
                                                                 
 dense_48 (Dense)            (None, 64)                8256      
                                                                 
 dropout_29 (Dropout)        (None, 64)               

In [ ]:
best_model_weights = './base.model'
checkpoint = ModelCheckpoint(
    best_model_weights,
    monitor = 'val_acc',
    verbose = 1,
    save_best_only= True,
    mode = 'min',
    save_weights_only= False,
    period = 1
)

callbacks = [checkpoint]
opt = optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = opt,
              loss = losses.sparse_categorical_crossentropy,
              metrics = ['acc'])

history = model.fit(X_train,y_train,validation_data = (X_val,y_val),
                   verbose = 1, callbacks=callbacks,epochs=10)

Epoch 1/10
959/963 [============================>.] - ETA: 0s - loss: 2.1747 - acc: 0.4365
Epoch 1: val_acc improved from inf to 0.45794, saving model to ./base.model
963/963 [==============================] - 6s 6ms/step - loss: 2.1738 - acc: 0.4367 - val_loss: 2.0156 - val_acc: 0.4579
Epoch 2/10
963/963 [==============================] - ETA: 0s - loss: 2.0462 - acc: 0.4570
Epoch 2: val_acc did not improve from 0.45794
963/963 [==============================] - 4s 4ms/step - loss: 2.0462 - acc: 0.4570 - val_loss: 1.9952 - val_acc: 0.4579
Epoch 3/10
953/963 [============================>.] - ETA: 0s - loss: 2.0285 - acc: 0.4587
Epoch 3: val_acc improved from 0.45794 to 0.45648, saving model to ./base.model
963/963 [==============================] - 5s 5ms/step - loss: 2.0299 - acc: 0.4581 - val_loss: 2.0058 - val_acc: 0.4565
Epoch 4/10
951/963 [============================>.] - ETA: 0s - loss: 2.0200 - acc: 0.4595
Epoch 4: val_acc did not improve from 0.45648
963/963 [================

In [ ]:
TrainLoss, Trainacc = model.evaluate(X_train,y_train)
TestLoss, Testacc = model.evaluate(X_test, y_test)
y_pred=model.predict(X_test)

57/57 [==============================] - 0s 2ms/step


In [ ]:
y_pred_bool = np.argmax(y_pred, axis=1)
rep = classification_report(y_test, y_pred_bool,output_dict = True)

In [ ]:
rep

{'0': {'precision': 0.28205128205128205,
  'recall': 0.0859375,
  'f1-score': 0.1317365269461078,
  'support': 128},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6},
 '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10},
 '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 101},
 '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11},
 '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4},
 '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3},
 '10': {'precision': 0.3640776699029126,
  'recall': 0.46875,
  'f1-score': 0.40983606557377045,
  'support': 160},
 '11': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7},
 '12': {'precision': 0.0, 'recall': 0.0, '